In [4]:
using Globtim
using CSV
using DataFrames

include("../src/lib_func.jl")

# Load the dataframe from the CSV file
df_2d = CSV.read("../data/camel_3_d6.csv", DataFrame)

# Constants and Parameters
d = 6      # Degree 
const n, a, b = 4, 5, 1 
const scale_factor = a / b  # Scaling constant, C is appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .1 , 2 / 10  # Sampling parameters
const tol_l2 = 1e-1               # Define the tolerance for the L2-norm


f = camel_3_by_3 # Objective function

camel_3_by_3 (generic function with 1 method)

In [5]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, n, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end

attained the desired L2-norm: 1.959486759860326e-14


In [6]:
using DynamicPolynomials, HomotopyContinuation, ProgressLogging, DataFrames
@polyvar(x[1:n]) # Define polynomial ring 
ap = main_nd(n, d, poly_approx.coeffs, x)
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x. 
PolynomialApproximant = sum(Float64.(ap) .* MonomialVector(x, 0:d))
grad = differentiate.(PolynomialApproximant, x)
sys = System(grad)

Check


System of length 4
 4 variables: x₁, x₂, x₃, x₄

 3.06050200626839e-13 + 100.000000000011*x₁ + 24.9999999999999*x₂ - 2.3704365869393e-13*x₃ - 7.66742442553705e-14*x₄ + 5.83445023305393e-13*x₂*x₁ - 2.55271207677904e-14*x₂*x₁^2 + 2.36949525354632e-13*x₂*x₁^3 + 4.76853894527721e-13*x₂*x₁^4 - 1.63350401558655e-14*x₂*x₃ + 2.08498210085108e-14*x₂*x₃^2 - 7.15002554290457e-15*x₂*x₃^3 + 3.3464512188227e-14*x₂*x₃^4 - 1.59663286924855e-12*x₂^2*x₁ - 1.41754938236056e-13*x₂^2*x₁^2 + 6.78149577749239e-13*x₂^2*x₁^3 - 4.74858571794318e-14*x₂^2*x₃ - 2.80836620953063e-14*x₂^2*x₃^2 - 1.84752159651815e-14*x₂^2*x₃^3 - 8.64308448585553e-13*x₂^3*x₁ - 2.77069158724068e-13*x₂^3*x₁^2 + 4.77001568321168e-14*x₂^3*x₃ - 8.56716562382656e-14*x₂^3*x₃^2 + 1.52991426565103e-12*x₂^4*x₁ + 5.95414627551087e-14*x₂^4*x₃ - 1.01360337401798e-12*x₃*x₁ + 1.14118903874245e-12*x₃*x₁^2 + 1.05042305202866e-12*x₃*x₁^3 - 6.85082474266706e-13*x₃*x₁^4 - 9.1445550997829e-12*x₃^2*x₁ - 2.12795604041463e-13*x₃^2*x₁^2 - 4.44148998081706e-12

Now we use Optim.jl package to initialize step 3 and converge to the local minimizers.
How do we know we have everything ? Use simple combinations of 

In [7]:
Real_sol_lstsq = HomotopyContinuation.solve(sys)
real_pts = HomotopyContinuation.real_solutions(Real_sol_lstsq; only_real=true, multiple_results=false)

Hasn't been modified yet.





Now we want to compute the distance between the critical points of the 2d function. We want to take the tensor product of coordinates of critical points recovered from the 2D case. Add a column for minimal distance of 2 first coordinates to original critical point, same for 2nd column. 

The true critical points are stored in df_2d. Need to split to distance to minima first coordinate and second set of coordinates ten combine them.


In [26]:
for i in 1:nrow(df)
    distances_1 = [norm([df.x[i], df.y[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    distances_2 = [norm([df.z[i], df.t[i]] - [df_2d.x[j], df_2d.y[j]]) for j in 1:nrow(df_2d)]
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2)    
    df.total_distance[i] = norm([df.x[i], df.y[i], df.z[i], df.t[i]] - [df_2d.x[min_index_1], df_2d.y[min_index_1], df_2d.x[min_index_2], df_2d.y[min_index_2]])
end
sorted_df = sort(df, :total_distance)

Row,x,y,z,t,col,val_f,total_distance,dist_to_loc_min,steps,converged
,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Int64,Bool
1,-1.74755,0.873776,3.93355e-13,6.56749e-14,6,0.298638,4.40152e-13,0.0,0,true
2,-1.07054,0.535271,3.89238e-13,8.63535e-14,6,0.877362,4.59623e-13,0.0,0,true
3,1.54971e-14,-1.23198e-13,3.8878e-13,1.00232e-13,6,3.65063e-25,4.64413e-13,0.0,0,true
4,1.07054,-0.535271,3.96089e-13,8.82379e-14,6,0.877362,5.37214e-13,0.0,0,true
5,1.74755,-0.873776,4.0445e-13,6.66781e-14,6,0.298638,6.59417e-13,0.0,0,true
6,1.89846e-14,-1.29946e-13,1.07054,-0.535271,6,0.877362,7.96295e-13,0.0,0,true
7,-1.07054,0.535271,1.07054,-0.535271,6,1.75472,8.08604e-13,0.0,0,true
8,-1.74755,0.873776,1.07054,-0.535271,6,1.176,8.09302e-13,0.0,0,true
9,1.07054,-0.535271,1.07054,-0.535271,6,1.75472,8.49068e-13,0.0,0,true


In [28]:
using LinearAlgebra
# Precompute df_2d coordinates
df_2d_coords = [df_2d.x df_2d.y]

# Create new columns for results
df.total_distance = Vector{Float64}(undef, nrow(df))

for i in 1:nrow(df)
    # Current coordinates
    coords_1 = [df.x[i], df.y[i]]
    coords_2 = [df.z[i], df.t[i]]

    # Compute distances to all points in df_2d
    distances_1 = vec(norm.(eachrow(df_2d_coords .- coords_1')))
    distances_2 = vec(norm.(eachrow(df_2d_coords .- coords_2')))

    # Find minimum distances and corresponding indices
    min_value_1, min_index_1 = findmin(distances_1)
    min_value_2, min_index_2 = findmin(distances_2)

    # Compute total distance
    min_coords_1 = df_2d_coords[min_index_1, :]
    min_coords_2 = df_2d_coords[min_index_2, :]
    df.total_distance[i] = norm([df.x[i], df.y[i], df.z[i], df.t[i]] - [min_coords_1; min_coords_2])
end

# Display the dataframe
sorted_df = sort(df, :val_f)
println(sorted_df)

25×10 DataFrame
 Row │ x             y             z             t             col    val_f        total_distance  dist_to_loc_min  steps  converged 
     │ Float64       Float64       Float64       Float64       Int64  Float64      Float64         Float64          Int64  Bool      
─────┼───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
   1 │  1.54971e-14  -1.23198e-13   3.8878e-13    1.00232e-13      6  3.65063e-25     4.64413e-13              0.0      0       true
   2 │  2.14315e-14  -1.36495e-13   1.74755      -0.873776         6  0.298638        1.34488e-12              0.0      0       true
   3 │ -1.74755       0.873776      3.93355e-13   6.56749e-14      6  0.298638        4.40152e-13              0.0      0       true
   4 │  9.78055e-15  -1.17944e-13  -1.74755       0.873776         6  0.298638        1.63269e-12              0.0      0       true
   5 │  1.74755      -0.873776      4.0445e-13    

In [21]:
for i in 1:nrow(df)
    # println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i], df.z[i], df.t[i]]
    #  + 0.2 * randn(Float64, 4)
    res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
    minimizer = Optim.minimizer(res)
    min_value = Optim.minimum(res)
    steps = res.iterations
    converged = Optim.converged(res)
    distance = norm(x0 - minimizer)
    df.dist_to_loc_min[i] = distance
    df.steps[i] = steps
    df.converged[i] = converged
    # summary(res)
end
print(df)

Iter     Function value   Gradient norm 
     0     5.972769e-01     1.164545e-09
 * time: 5.984306335449219e-5
Iter     Function value   Gradient norm 
     0     1.176000e+00     1.279950e-09
 * time: 5.507469177246094e-5
Iter     Function value   Gradient norm 
     0     1.176000e+00     1.154054e-09
 * time: 2.5987625122070312e-5
Iter     Function value   Gradient norm 
     0     1.754723e+00     3.425229e-11
 * time: 3.1948089599609375e-5
Iter     Function value   Gradient norm 
     0     1.754723e+00     3.425229e-11
 * time: 2.4080276489257812e-5
Iter     Function value   Gradient norm 
     0     1.176000e+00     1.154054e-09
 * time: 2.384185791015625e-5
Iter     Function value   Gradient norm 
     0     8.773616e-01     1.712615e-11
 * time: 1.7881393432617188e-5
Iter     Function value   Gradient norm 
     0     5.972769e-01     1.196019e-09
 * time: 2.193450927734375e-5
Iter     Function value   Gradient norm 
     0     1.176000e+00     1.154054e-09
 * time: 1.8119812